# Create the inference wrapper
In order to use the trained model, you need to write a wrapper script that loads the model and feeds it the data coming from the configured data source.
An example is already implemented under the `src` folder and in the `entrypoint.py` file. This notebook helps you to write such a wrapper, and understand how it works.

To execute this notebook, you need:
- the model created in notebook [10-CreateClassificationModel](10-CreateClassificationModel.ipynb)

## Load the model
The wrapper script must load the model first.
The filesystem layout (python scripts in the `src` folder, model in the `models` folder) is reproduced during pipeline run,
so you can run the same code while experimenting.

In [ ]:
import tensorflow as tf
from pathlib import Path
%matplotlib inline

model_path = Path('../models/classification_mobilnet.tflite').resolve()

with open(model_path, 'rb') as rpl:
    model = rpl.read()

interpreter = tf.lite.Interpreter(model_content=model)
interpreter.allocate_tensors()


## Feed image into the model
Once the model is loaded it is ready to make predictions but some preprocessing of the images is still needed. The following code bit shows an example of wrapping the preprocessing and the predicting steps together.

In [ ]:
import numpy as np


# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
SCALE = 255

# Define a method to wrap the preprocessing of the image and the model making a prediction
def predict_from_image(image):
    
    input_arr = np.array(image)*1/SCALE
    assert input_arr.shape == (IMAGE_WIDTH, IMAGE_HEIGHT, 3), "The input image must contain RGB channels without alpha and must be of a certain size."
    input_arr = np.array([input_arr], dtype=np.float32)  # Convert single image to a batch.

    interpreter.set_tensor(input_details[0]['index'], input_arr)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_details[0]['index'])

    index = np.argmax(predictions, axis=-1).item()
    return index, float(predictions[0][index])


Let's try it with an example.

In [ ]:
from matplotlib import pyplot as plt
from pathlib import Path

image_dir = Path('../data/processed')

images = list(str(f) for f in image_dir.rglob("*.jpg"))
images_count = len(images)

In [ ]:
import cv2

image = cv2.imread(images[0])  # BGR image 224x224x3
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # RGB image 224x224x3
image = cv2.resize(image, (224, 224))

plt.imshow(image)

In [ ]:
predict_from_image(image)

### Create ImageSet payload format

The method defined below creates the required format, and will be put into the payload with name `vision_payload` which will be provided by the `AI Inference Server` when an image is arrived from the selected camera through `Vision Connector Application`.

In [ ]:
def image_to_bayer(image_path):
    im = cv2.imread(image_path)  
    im = cv2.resize(im, (224, 224)) # RGB image 224x224x3
    (height, width) = im.shape[:2]
    (R,G,B) = cv2.split(im)

    bayerrg8 = np.zeros((height, width), np.uint8)

    # strided slicing for this pattern:
    #   R G
    #   G R
    bayerrg8[0::2, 0::2] = R[0::2, 1::2] # top left
    bayerrg8[0::2, 1::2] = G[0::2, 0::2] # top right
    bayerrg8[1::2, 0::2] = G[1::2, 1::2] # bottom left
    bayerrg8[1::2, 1::2] = B[1::2, 0::2] # bottom right

    return bayerrg8, width, height

In [ ]:
import datetime

def create_imageset_dict(image_path):
    timestamp = datetime.datetime.now()
    
    image_bytes, width, height = image_to_bayer(image_path)

    return {
        "version": "1",  # version of the Metadata format
        "count": 1,  # Number of images on message
        "timestamp": timestamp.isoformat(),  # Camera acquisition time
        "detail": [{  # list of images with detailed information
            "id": str(image_path),  # unique image identifier. this case we use the filename of the original image
            "timestamp": str(timestamp.timestamp()),  # Timestamp provided by the camera
            "width": width,  # image width
            "height": height,  # image height
            "format": "BayerRG8",  # image format configure
            "metadata": "",  # optional extra information on image
            "image": image_bytes  # image binary with the given 'format'
        }]
    }

image_set_payload = {"vision_payload": create_imageset_dict(images[0])}
image_set_payload

### Extract image from ImageSet

Now the task of the PythonComponent is to extract image data from the payload, and creates a flat array of float32 data.  
This time the original image is packaged into the payload in `BayerRG8` format, so the Python component converts it to a one-dimensional float32 array with using PIL Image and numpy transformations.

In [ ]:
extracted = image_set_payload['vision_payload']
image_detail = extracted["detail"][0]

iuid = image_detail['id']
width = image_detail.get("width", 224)
height = image_detail.get("height", 224)

image_data = np.frombuffer(image_detail['image'], dtype=np.uint8)  # BayerRG8, (width x height, )
print(image_data.shape)
image_data = image_data.reshape(width, height)                           # BayerRG8, (width, height)
print(image_data.shape)
image_data = cv2.cvtColor(image_data, cv2.COLOR_BayerRG2RGB)             # RGB, (width, height, 3)
print(image_data.shape)

image_array = image_data.astype(np.float32) / 255.  # normalizing into [0,1) range and converting to float32

print(f"image_array shape: {image_array.shape}")  # checking the image dimensions
plt.imshow(image_array)  # showing the image
plt.axis('off')

image_array = image_array.transpose(2,0,1)  # changing the shape from (224, 224, 3) to (3, 224, 224) as expected by the model
inputs = np.array(image_array.ravel())  # flattening the 3 dimensional array and adding to an empty batch
print(f"inputs shape {inputs.shape} and type '{inputs.dtype}'")  # checking the input shape and type

## Create entrypoint

Finally, we are ready to wrap everything together in an entrypoint script that the AI Inference Server can execute.

In [ ]:
def process_input(data: dict):
    extracted = data['vision_payload']
    image_detail = extracted["detail"][0]

    iuid = image_detail['id']
    width = image_detail.get("width", 224)
    height = image_detail.get("height", 224)

    image_data = np.frombuffer(image_detail['image'], dtype=np.uint8)
    image_data = image_data.reshape(width, height)                         
    
    image_data = cv2.resize(image_data, (224, 224))
    image = cv2.cvtColor(image_data, cv2.COLOR_BayerRG2RGB)

    if image is None:
        return None
    
    prediction, _ = predict_from_image(image)

    return {"prediction": str(prediction)}


Let's try it with an example.

In [ ]:
image_set_payload = create_imageset_dict('../data/processed/simatic_photos/S7_1500/IMG_1651.JPG')
data = { "vision_payload": image_set_payload }

print(process_input(data))

## Metrics and Object Output

You can return multiple outputs, and some of those outputs can serve as custom metrics for your running model.

When returning such custom metric outputs, the int or float value must be serialized in a specific way, like in the example below.

The metrics have to be defined separately from the outputs when creating a pipeline component.

The latest versions of AI Inference Servers support an internal variable type which can be transferred between pipeline steps or can be passed to a Zero Message Queue.

In [ ]:
import json

def process_input(data: dict):

    extracted = data['vision_payload']
    image_detail = extracted["detail"][0]

    width = image_detail.get("width", 224)
    height = image_detail.get("height", 224)

    image_data = np.frombuffer(image_detail['image'], dtype=np.uint8)
    image_data = image_data.reshape(width, height)                         
    
    image_data = cv2.resize(image_data, (224, 224))
    image = cv2.cvtColor(image_data, cv2.COLOR_BayerRG2RGB)
    
    if image is None:
        return None

    prediction, ic_probability = predict_from_image(image)

    return {
        "prediction": str(prediction),
        "ic_probability": metric_output(ic_probability)
    }

def metric_output(v: float):
    return json.dumps({"value": v})


In [ ]:
result = process_input(data)

print("prediction:", result["prediction"])
print("probability:", result["ic_probability"])